# How do CNN actually work ?

## Introduction

Neural network enthusiasm started many years ago at Cornell University. It was in 1957 that Frank Rosenblatt came up with the perceptron. It was a simple, one neuron, network that learned the decision function between two linearly separable classes. Discarded until the 1980s, deep learning came back to light when scientists got interested in the backpropagation algorithm again. But it was later, in the 2010s, that deep learning truly became feasible, thanks to the increase in computational power and the huge amount of available data.  
Nowadays, CNN is the go to model for image classification. Interest in CNN grew when AlexNet dominated the competition in 2012 during the ImageNet classification challenge. This kind of network came as an evolution from the fully connected layers. But how do they actually work ? And why is it much better than fully connected networks at tasks such as image classification ?

## Fully connected networks
 
A fully connected network is a model where each neuron in one layer is connected to all neurons in the next layer. Fully connected layers are separated by non-linear functions transformations such as sigmoid, or ReLU. This architecture allows fully connected networks to be "universal approximators" meaning that they can theoretically approximate any function.  
A fully connected layer implements the following function, for an input $x$:
$$f(x) = x^T W + b$$
Where $W$ is the weight matrix and $b$ is the bias vector. A FC layer implements a linear transformation.  
The intuition behind it is that a fully connected layer implements a huge filter on the input. Theoretically it could learn any function but practically it requires a lot of computation and often yields poor results.  
Images are structured data. That means that for one pixel, the neighboring pixels are closely related but the pixels far away probably have little to no link. A fully connected layer "wastes" resources trying to learn relations between far away pixels which are not related. Hence for a 2D image of size $100 \times 100$ and $10$ output features, there are $100000$ parameters. Most of them will have little importance. This kind of layer requires a lot of unneeded computation when applied to images.  

#### Forward propagation
  
For a 2d vector input $x$ and 2d vector output $o$:
  
$$o_1 = w_{11} \times x_1 + w_{12} \times x_2$$
$$o_2 = w_{21} \times x_1 + w_{22} \times x_2$$
We can represent this in matrix form:
$$\begin{bmatrix}
x_1 & x_2
\end{bmatrix} \times 
\begin{bmatrix}
w_{11} & w_{21} \\
w_{12} & w_{22}
\end{bmatrix} = \begin{bmatrix}
o_1 \\
o_2
\end{bmatrix}
$$
With a lighter notation:
$$x^T W = o$$
This is represented in the following picture:  
![2 neurons layers](images/2_neurons_linear.png)  
Now when the input is 2d (ex: an image), we first need to flatten it to feed to a linear layer. You may wonder why since the matrix product is perfectly doable. However the purpose of a linear layer is to connect each neuron in one layer to all the neurons in the next layer. If the input is not flattened, only the row elements will be connected together, hence we won't be able to mutualize the information at the image level. This is explained in the following picture:  
![2d linear input](images/2d_linear_input.png)  
Here the math works except $o_{11} = w_{11} \times x_11 + w_{21} \times x_12$, so $o_{11}$ only sees a row ! Actually each element of the output matrix only mutualizes information in a single row. In images, there is a strong correlation between adjacent rows, hence the matrix representation for the input is not optimal. This is why we need to flatten the image before processing:  
![2d linear input](images/flattened_linear_input.png)  
Or following the first picture schema:  
![2d linear input](images/flattened_linear_input_schema.png)  
The linear layer is then able to mutualize the information across the whole image instead of a single row. So when you work with an image of size $m \times n$ and want $f$ output features, the linear layer is a matrix of size $mn \times f$.  
  
#### Backward propagation

***The details for this part are available in the annex***

The backward propagation consists in updating the parameters $W$ and $b$ in order to minimize the loss of the network $L$. We get the gradient of $L$ with respect to the output of the linear layer as input of the backward propagation $\nabla L$. Using SGD with parameter $\alpha$:
$$W_{t} = W_{t-1} + \alpha \frac{\partial L}{\partial W}$$
$$b_{t} = b_{t-1} + \alpha \frac{\partial L}{\partial b}$$
Where:
$$\frac{\partial L}{\partial W} = \nabla L^T x$$
$$\frac{\partial L}{\partial b} = \nabla L^T$$
  
#### Conclusion
  
This looks like a scalar product between the image and a series of weight vectors. Actually each column in the weight matrix can be considered as a filter the size of the whole image. This is totally suboptimal and a waste of resources, but this is the only way to process images with a linear layer. This kind of computation was the standard in Deep Learning until AlexNet and the avent of convolutional layers.

## Convolutions
 
Unlike a fully connected layer, a convolution layer does not connect every neurons from one layer to the next. It uses small filters that process data in a window. This means filters don't care about far away pixels. Instead of learning one huge filter, a convolutional layer learns many small filters. That makes the filtering shift invariant, an object could be anywhere in the image and still detected by the filter. On the other hand, fully connected layers are not shift invariant. A shift in the image while non-problematic for a human or CNN is a challence for a FC network. This is why it needs to learn a very general filter which is often impossible to converge.  
For each pixel $(x, y)$ of the input image $I$ of size $(W, H)$, the output will be:
$$CONV(I_{x,y},K) = \sum_{i=-n_w}^{n_w} \sum_{j=-n_h}^{n_h} I_{x+i, y+j} K_{i,j}$$
Where $K$ is the convolution kernel of size $(i, j)$  
*Note: the size of the output image will be $(W - 2 (n_w//2), H - 2 (n_h//2))$. We can pad the borders with 0 so the size of the output image is the same as the input image. This technic is called **padding***.    

#### Forward propagation

Unlike linear layers, convolutional layers are nativally able to mutualize information across the rows of an image (without any pre-processing). The breakthrough was to connect only a small subset of neurons of the layer to each neuron in the next layer and **share** the weights:  
![2d conv_input](images/convolutional_layer_schema_1.png)  
![2d conv_input](images/convolutional_layer_schema_2.png)  
These weights can be represented in a matrix. It is basically as if each page of the matrix is the input of a linear layer like we saw in the previous section. In the case of CNN the weight matrix is called a **kernel**.  
The fact that these kernel are small (usually 3x3) grants a nice property:  
- Shift invariance: since the weights are shared and the patch is small, it does not matter if an object is translated by a few pixels, the kernel will still be able to detect it easily. This was not the case for linear layers that learnt a large filter, where a small shift in the image would change the filter's output.
However, CNN are not scale nor rotation invariant (though they are tolerant to scale change up to a certain limit).

#### Backward propagation
  
***The details for this part are available in the annex***
  
Ok this is the hard part, so please bear with me, everything is going to go fine.  
Let's keep the case of a 2x2 kernel going across a 3x3 image for simplicity. The forward propagation is:
$$o_{11} = w_{11}x_{11} + w_{12}x_{12} + w_{21}x_{21} + w_{22}x_{22}$$
$$o_{12} = w_{11}x_{12} + w_{12}x_{13} + w_{21}x_{22} + w_{22}x_{23}$$
$$o_{21} = w_{11}x_{21} + w_{12}x_{22} + w_{21}x_{31} + w_{22}x_{32}$$
$$o_{22} = w_{11}x_{22} + w_{12}x_{23} + w_{21}x_{32} + w_{22}x_{33}$$
The derivative for $w_{11}$ is:
$$\frac{\partial L}{\partial w_{11}} = 
\frac{\partial L}{\partial o_{11}} x_{11} + 
\frac{\partial L}{\partial o_{12}} x_{12} + 
\frac{\partial L}{\partial o_{21}} x_{21} + 
\frac{\partial L}{\partial o_{22}} x_{22}$$
So in order to get $\frac{\partial L}{\partial W}$ we perform a convolution:  
![2d conv_input](images/backward_conv.png)